# remove the combined mask at first 


In [18]:
import os
import glob 
combined_scan = glob.glob('New/s*/*/combined*')
print(len(combined_scan ))
# remove these files
for file in combined_scan:
    os.remove(file)
combined_scan = glob.glob('New/s*/*/combined*')
print(len(combined_scan ))

11
0


In [15]:
combined_scan = glob.glob('New/s0001/segmentations/*')
len(combined_scan)

117

In [16]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('label_names.xlsx')

# Extract the multiplied_labels column
multiplied_labels = df['multiplied_labels'].tolist()

# Check for duplicates
duplicates = [label for label in multiplied_labels if multiplied_labels.count(label) > 1]
if duplicates:
    print("Duplicate labels found:", set(duplicates))
else:
    print("No duplicate labels found.")

# Check the maximum label value
max_label = max(multiplied_labels)
print("Maximum label value:", max_label)

# Check the number of unique labels
unique_labels = set(multiplied_labels)
print("Number of unique labels:", len(unique_labels))


No duplicate labels found.
Maximum label value: 117
Number of unique labels: 117


In [2]:
import pandas as pd
import glob
import nibabel as nib
import numpy as np
import tqdm
import os 

# Load label information from Excel
df = pd.read_excel('label_names.xlsx')
label_names = df['label_names'].tolist()
multiplied_labels = df['multiplied_labels'].tolist()
print(label_names[0], len(label_names))
print(multiplied_labels[0], len(multiplied_labels))

# Get list of folders containing masks
folders = glob.glob('New/s*/se*')
folders.sort()

for folder_num, folder in enumerate(tqdm.tqdm(folders, desc="Processing folders")):
    # Load one mask to get the shape and affine information
    initial_file = os.path.join(folder, label_names[0])
    initial_mask_nii = nib.load(initial_file)
    mask_shape = initial_mask_nii.get_fdata().shape
    affine = initial_mask_nii.affine

    # Initialize the combined mask with zeros
    combined_mask = np.zeros(mask_shape)
    non_empty_label_count = 0
    num_masks = []

    for label_num, (label_name, label_value) in enumerate(zip(label_names, multiplied_labels)):
        file_path = os.path.join(folder, label_name)
        comb_file_path = os.path.join(folder, f'combined_mask.nii.gz')

        file = nib.load(file_path)
        mask = file.get_fdata()
        combined_scan = nib.load(comb_file_path).get_fdata()

        
        
        if mask.sum() == 0:
            continue
        else:
            non_empty_label_count += 1
            num_masks.append(label_value)
    
    #print(num_masks)
    #print(np.unique(combined_scan))
    # get the unique values in between combined mask and num_masks
    unique_values = np.unique(combined_scan)
    unique_values = np.setdiff1d(unique_values, num_masks)
    #print()
    
    if len(unique_values) > 1:
        print(folder, len(np.unique(combined_scan)), unique_values)
    #print(non_empty_label_count, len(np.unique(combined_scan)), unique_values)

        


adrenal_gland_left.nii.gz 117
1 117


Processing folders:   0%|          | 0/1228 [00:00<?, ?it/s]

Processing folders: 100%|██████████| 1228/1228 [7:11:35<00:00, 21.09s/it]  


In [15]:
len(glob.glob('dataset_nii/val/labels/*'))

89

In [12]:
len(file)

1171

In [8]:
import os 
import nibabel as nib
import numpy as np
import glob
from tqdm import tqdm

file = glob.glob('dataset_nii/*/labels/*')
for f in tqdm(file):
    mask = nib.load(f).get_fdata()
    number = np.unique(mask)
    if 117 in number:
        print(f, number)

100%|██████████| 1171/1171 [08:58<00:00,  2.17it/s]


In [24]:
import pandas as pd
df = pd.read_excel('meta.xlsx')
df1 = df[df['split']=='train']
print(len(df1))
df2 = df[df['split']=='val']
print(len(df2))
len(df1) + len(df2)
df['split'].value_counts()

1082
57


split
train    1082
test       89
val        57
Name: count, dtype: int64

# Devide the datset based on split column value 'train' or 'test'



In [25]:
# read the mketa.xlsx file 
import pandas as pd
df = pd.read_excel('meta.xlsx')
# select only train values in split column
df = df[df['split']=='train']
print(df.columns)
# make directories for train and test
import os
os.makedirs('dataset_nii/train/images', exist_ok=True)
os.makedirs('dataset_nii/train/labels', exist_ok=True)


# get the name of images and labels
files = df['image_id'].values
# from these folders get the images and labels
image_folder = 'New'
label_folder = 'segmentations' 
import glob 
import tqdm
for i in tqdm.tqdm(range(len(files))):

    images = 'New/'+files[i]+'/ct.nii.gz'
    labels = 'New/'+files[i]+'/segmentations/combined_mask.nii.gz'
    # copy the images, change the name of the images sand then save them in the train folder
    import shutil
    shutil.copy(images, 'dataset_nii/train/images/'+files[i]+'.nii.gz') 
    shutil.copy(labels, 'dataset_nii/train/labels/'+files[i]+'.nii.gz')




Index(['image_id', 'age', 'gender', 'institute', 'study_type', 'split',
       'manufacturer', 'scanner_model', 'kvp', 'pathology',
       'pathology_location'],
      dtype='object')


100%|██████████| 1082/1082 [00:13<00:00, 82.04it/s] 


In [26]:
# read the mketa.xlsx file 
import pandas as pd
df = pd.read_excel('meta.xlsx')
# select only train values in split column
df = df[df['split']=='test']
print(df.columns)
# make directories for train and test
import os

os.makedirs('dataset_nii/test/images', exist_ok=True)
os.makedirs('dataset_nii/test/labels', exist_ok=True)

# get the name of images and labels
files = df['image_id'].values
# from these folders get the images and labels
image_folder = 'New'
label_folder = 'segmentations' 
import glob 
import tqdm
for i in tqdm.tqdm(range(len(files))):

    images = 'New/'+files[i]+'/ct.nii.gz'
    labels = 'New/'+files[i]+'/segmentations/combined_mask.nii.gz'
    # copy the images, change the name of the images sand then save them in the train folder
    import shutil
    shutil.copy(images, 'dataset_nii/test/images/'+files[i]+'.nii.gz') 
    shutil.copy(labels, 'dataset_nii/test/labels/'+files[i]+'.nii.gz')




Index(['image_id', 'age', 'gender', 'institute', 'study_type', 'split',
       'manufacturer', 'scanner_model', 'kvp', 'pathology',
       'pathology_location'],
      dtype='object')


100%|██████████| 89/89 [00:01<00:00, 63.75it/s]


In [27]:
# read the mketa.xlsx file 
import pandas as pd
df = pd.read_excel('meta.xlsx')
# select only train values in split column
df = df[df['split']=='val']
print(df.columns)
# make directories for train and test
import os

os.makedirs('dataset_nii/val/images', exist_ok=True)
os.makedirs('dataset_nii/val/labels', exist_ok=True)

# get the name of images and labels
files = df['image_id'].values
# from these folders get the images and labels
image_folder = 'New'
label_folder = 'segmentations' 
import glob 
import tqdm
for i in tqdm.tqdm(range(len(files))):

    images = 'New/'+files[i]+'/ct.nii.gz'
    labels = 'New/'+files[i]+'/segmentations/combined_mask.nii.gz'
    # copy the images, change the name of the images sand then save them in the train folder
    import shutil
    shutil.copy(images, 'dataset_nii/val/images/'+files[i]+'.nii.gz') 
    shutil.copy(labels, 'dataset_nii/val/labels/'+files[i]+'.nii.gz')




Index(['image_id', 'age', 'gender', 'institute', 'study_type', 'split',
       'manufacturer', 'scanner_model', 'kvp', 'pathology',
       'pathology_location'],
      dtype='object')


100%|██████████| 57/57 [00:00<00:00, 79.91it/s]


In [13]:
print(len('datase_nii/train/images/') == len('datase_nii/train/labels/'), len('datase_nii/test/images/') == len('datase_nii/test/labels/'))

True True


# check images 

In [1]:
import numpy as np
from PIL import Image
import cv2

def convert_mask_to_segmentation_(mask_path, output_txt_path, class_mapping):
    # Load the mask image
    mask_image = Image.open(mask_path)
    
    # Ensure the mask is in integer mode
    if mask_image.mode not in ['L', 'I']:
        mask_image = mask_image.convert('I')
    
    # Convert the mask image to a numpy array
    mask_np = np.array(mask_image).astype(np.int32)
    
    # Identify unique labels (exclude 0 if it's background)
    unique_labels = np.unique(mask_np)
    unique_labels = unique_labels[unique_labels != 0]
    
    # Image dimensions for normalization
    height, width = mask_np.shape[:2]
    
    # Open the output file
    with open(output_txt_path, 'w') as f:
        for label in unique_labels:
            # Verify the label exists in class_mapping
            if label not in class_mapping:
                print(f"Warning: Label {label} not found in class mapping.")
                continue
            
            # Create a binary mask for the current label
            mask_bin = (mask_np == label).astype(np.uint8) * 255  # Ensure binary mask with 0 and 255
            
            # Determine OpenCV version for findContours compatibility
            if int(cv2.__version__.split('.')[0]) >= 4:
                contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            else:
                _, contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            for contour in contours:
                if len(contour) < 3:
                    continue  # Skip if contour has less than 3 points

                # Reshape and normalize the contour coordinates
                contour = contour.reshape(-1, 2).astype(float)
                contour[:, 0] = contour[:, 0] / width  # Normalize x
                contour[:, 1] = contour[:, 1] / height  # Normalize y
                segmentation = contour.flatten().round(6).tolist()
                
                # Format segmentation points as strings
                segmentation_str = ' '.join(f"{coord:.6f}" for coord in segmentation)
                class_id = class_mapping[label]
                
                # Write to file in format: <class_id> <points>
                f.write(f"{class_id} {segmentation_str}\n")


In [2]:

class_mapping = {0: 1, 1: 2, 2: 3, 3: 4, 4:5, 5:6, 6:7, 7:8, 8:9, 9:10, 10:11, 11:12, 12:13, 13:14, 14:15, 15:16, 16:17, 17:18, 18:19, 19:20, 20:21, 21:22, 22:23, 23:24, 
                 24:25, 25:26, 26:27, 27:28, 28:29, 29:30, 30:31, 31:32, 32:33, 33:34, 34:35, 35:36, 36:37, 37:38, 38:39, 39:40, 40:41, 41:42, 42:43, 43:44, 44:45, 45:46, 
                 46:47, 47:48, 48:49, 49:50, 50:51, 51:52, 52:53, 53:54, 54:55, 55:56, 56:57, 57:58, 58:59, 59:60, 60:61, 61:62, 62:63, 63:64, 64:65, 65:66, 66:67, 67:68, 
                 68:69, 69:70, 70:71, 71:72, 72:73, 73:74, 74:75, 75:76, 76:77, 77:78, 78:79, 79:80, 80:81, 81:82, 82:83, 83:84, 84:85, 85:86, 86:87, 87:88, 88:89, 89:90, 90:91,
                   91:92, 92:93, 93:94, 94:95, 95:96, 96:97, 97:98, 98:99, 99:100, 100:101, 101:102, 102:103, 103:104, 104:105, 105:106, 106:107, 107:108, 108:109, 109:110, 110:111, 
                   111:112, 112:113, 113:114, 114:115, 115:116, 116:117, 117:118, 118:119, 119:120, 120:121, 121:122, 122:123, 123:124, 124:125, 125:126, 126:127, 127:128, 128:129, 
                   129:130, 130:131, 131:132, 132:133, 133:134, 134:135, 135:136, 136:137, 137:138, 138:139, 139:140, 140:141, 141:142, 142:143, 143:144, 144:145, 145:146, 146:147, 
                   147:148, 148:149, 149:150, 150:151, 151:152, 152:153, 153:154, 154:155, 155:156, 156:157, 157:158, 158:159, 159:160, 160:161, 161:162, 162:163, 163:164, 164:165, 
                   165:166, 166:167, 167:168, 168:169, 169:170, 170:171, 171:172, 172:173, 173:174, 174:175, 175:176, 176:177, 177:178, 178:179, 179:180, 180:181, 181:182, 182:183,
                     183:184, 184:185, 185:186, 186:187, 187:188, 188:189, 189:190, 190:191, 191:192, 192:193, 193:194, 194:195, 195:196, 196:197, 197:198, 198:199, 199:200, 200:201}



In [2]:
import glob 
file = glob.glob('dataset/train/labels/*')
file[0]

'dataset/train/labels/s0647.nii.gz_slice_227_0_.txt'

In [4]:
mask_path = 'dataset/train/labels/s0001.nii.gz'
convert_mask_to_segmentation_(file[0], 'output1_txt_path.txt', class_mapping)

In [8]:
# check text file. 
import pandas as pd
df = pd.read_csv('output1_txt_path.txt', header=None, sep=' ')

ParserError: Error tokenizing data. C error: Expected 83 fields in line 5, saw 157


In [7]:
df

,6 0.606557 0.148649 0.598361 0.162162 0.590164 0.162162 0.573770 0.189189 0.573770 0.202703 0.557377 0.229730 0.557377 0.243243 0.540984 0.270270 0.540984 0.283784 0.532787 0.297297 0.532787 0.324324 0.524590 0.337838 0.532787 0.351351 0.532787 0.378378 0.540984 0.391892 0.540984 0.405405 0.557377 0.432432 0.565574 0.432432 0.573770 0.445946 0.590164 0.445946 0.598361 0.459459 0.655738 0.459459 0.663934 0.445946 0.680328 0.445946 0.688525 0.432432 0.696721 0.432432 0.704918 0.418919 0.713115 0.418919 0.721311 0.405405 0.729508 0.405405 0.754098 0.364865 0.754098 0.283784 0.762295 0.270270 0.762295 0.229730 0.737705 0.189189 0.721311 0.189189 0.713115 0.175676 0.688525 0.175676 0.680328 0.162162 0.631148 0.162162 0.622951 0.148649
0,7 0.614754 0.621622 0.606557 0.635135 0.590164...
1,84 0.803279 0.432432 0.778689 0.472973 0.77868...
2,98 0.926230 0.364865 0.918033 0.378378 0.91803...
3,99 0.803279 0.243243 0.778689 0.283784 0.77868...
